In [1]:
from API_utils import fetch_from_API, show_dates, show_files_for_site_date
import os
import numpy as np

For now we will just do TEAKs oldest Lidar flight

In [2]:
site = 'TEAK'
productcode = 'DP1.30003.001'
data_path = '/home/jovyan/tmp'

os.makedirs(data_path, exist_ok=True)

#show_dates(site, productcode)


In [6]:
files = show_files_for_site_date(productcode, site, '2018-06')
laz = []
for file in files:
    if 'full_boundary.kml' in file['name']:
        boundaries = file
    if 'classified_point_cloud_colorized.laz' in file['name']:
        laz.append(file)

In [9]:
import requests
import hashlib


def download_from_NEON_API(f, data_path):

    attempts = 0 
    while attempts < 4:
        try:
            # get the file 
            handle = requests.get(f['url'])
            
            #check the md5 if it exists
            if f['md5']:
                md5 = hashlib.md5(handle.content).hexdigest()
                if md5 == f['md5']:
                    success = True
                    attempts = 4
                else:
                    fmd5 = f['md5']
                    print(f'md5 mismatch on attempt {attempts}')
                    success = False
                    attempts = attempts + 1
            else: 
                success = True
                attempts = 4
        except Exception as e:
            print(f'Warning:\n{e}')
            success = False
            attempts = attempts + 1
    # write the file
    if success:
        fname = os.path.join(data_path, f['name'])
        with open(fname, 'wb') as sink:
            sink.write(handle.content)
    else:
        raise Exception('failed to download')

In [10]:
download_from_NEON_API(boundaries, data_path)

In [12]:
from dask import delayed, compute
from dask.diagnostics import ProgressBar

results = []
for f in laz:
    results.append(delayed(download_from_NEON_API)(f, data_path))

with ProgressBar():
    computed = compute(*results)

[########################################] | 100% Completed |  1min 51.3s


We need to isntall pdal to process the laz files

In [13]:
!pip install pdal

     |████████████████████████████████| 945 kB 5.7 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for pdal: filename=PDAL-2.3.7-cp38-cp38-linux_x86_64.whl size=325486 sha256=2e225a46bde0ed759e430815fdb3227bcbe0c6843c05239109c10494f0a8486d
  Stored in directory: /home/jovyan/.cache/pip/wheels/49/48/b4/3f9a021e4d674d9aa44dd61989d53d3add21ad6d69b288235a
Successfully built pdal


In [14]:
import pdal

ValueError: numpy.ndarray size changed, may indicate binary incompatibility. Expected 88 from C header, got 80 from PyObject

In [ ]:
def make_pipe(bbox, chm_path, srs='EPSG:3857', threads=4, resolution=1):
    '''Creates, validates and then returns the pdal pipeline
    
    Arguments:
    bbox       -- Tuple - Bounding box in srs coordintes (default srs is EPSG:3857),
                  in the form: ([xmin, xmax], [ymin, ymax]).
    chm_path   -- String - Path where the CHM shall be saved. Must include .tif exstension.
    srs        -- String - EPSG identifier for srs  being used. Defaults to EPSG:3857
                  because that is what ept files tend to use.
    threads    -- Int - Number os threads to be used by the reader.ept. Defaults to 4.
    resolution -- Int or Float - resolution (m) used by writers.gdal
    '''
    
    t = Template('''
    {
        "pipeline": [
            {
            "bounds": "${scaled_bbox}",
            "filename": "https://s3-us-west-2.amazonaws.com/usgs-lidar-public/USGS_LPC_CA_LosAngeles_2016_LAS_2018/ept.json",
            "type": "readers.ept",
            "tag": "readdata",
            "spatialreference": "${srs}",
            "threads": "${threads}"
            },
            {
            "type":"filters.outlier",
            "method":"radius",
            "radius":1.0,
            "min_k":4
            },
            {
            "type":"filters.range",
            "limits":"returnnumber[1:1]"
            },
            {
            "type": "filters.reprojection",
            "in_srs":"${srs}",
            "out_srs": "EPSG:26911"
            },
            {
            "type":"filters.smrf",
            "scalar":1.2,
            "slope":0.2,
            "threshold":0.45,
            "window":16.0
            },
            {
            "type":"filters.hag_delaunay"
            },
            {
            "type":"filters.range",
            "limits":"HeightAboveGround[0:35]"
            },
            {
            "filename": "${chm_path}",
            "gdalopts": "tiled=yes,     compress=deflate",
            "nodata": -9999,
            "output_type": "idw",
            "resolution": 1,
            "type": "writers.gdal",
            "window_size": 6,
            "dimension": "HeightAboveGround"
            }
        ]
    }''')

    pipe = t.substitute(scaled_bbox=bbox, srs=srs, chm_path=chm_path, threads=threads)
    pipeline = pdal.Pipeline(pipe)
    if pipeline.validate():
        return(pipeline)
    else:
        raise Exception('Bad pipeline (sorry to be so ambigous)!')